### Importação das bibliotecas e download do Dataset

In [13]:
#Instale o Kaggle API se necessário
!pip install kaggle

In [15]:
!kaggle datasets download -d blastchar/telco-customer-churn

import zipfile
with zipfile.ZipFile("telco-customer-churn.zip", "r") as z:
    z.extractall(".")

Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
License(s): copyright-authors
telco-customer-churn.zip: Skipping, found more recently modified local copy (use --force to force download)


In [17]:
# Importação das bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [18]:
#Carregamento do Dataset
data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [19]:
print(data.head())
print(data.info())
print(data['Churn'].value_counts())

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

### Pré-processamento dos dados

In [22]:
# Converter a coluna 'Churn' para binário
data['Churn'] = data['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# Selecionar features numéricas e categóricas
numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
categorical_features = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
                        'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                        'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
                        'PaperlessBilling', 'PaymentMethod']

# Tratar dados faltantes
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data.fillna(data.mean(), inplace=True)

# Codificar variáveis categóricas
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

# Separar as features (X) e o target (y)
X = data.drop(columns=['Churn', 'customerID'])
y = data['Churn']

TypeError: Could not convert ['7590-VHVEG5575-GNVDE3668-QPYBK7795-CFOCW9237-HQITU9305-CDSKC1452-KIOVK6713-OKOMC7892-POOKP6388-TABGU9763-GRSKD7469-LKBCI8091-TTVAX0280-XJGEX5129-JLPIS3655-SNQYZ8191-XWSZG9959-WOFKT4190-MFLUW4183-MYFRB8779-QRDMV1680-VDCWW1066-JKSGK3638-WEABW6322-HRPFA6865-JZNKO6467-CHFZW8665-UTDHZ5248-YGIJN8773-HHUOZ3841-NFECX4929-XIHVW6827-IEAUQ7310-EGVHZ3413-BMNZE6234-RAAPL6047-YHPVI6572-ADKRS5380-WJKOV8168-UQWWF8865-TNMNX9489-DEDVP9867-JCZSP4671-VJLCL4080-IIARD3714-NTNFO5948-UJZLF7760-OYPDY7639-LIAYI2954-PIBKO8012-SOUDQ9420-LOJKX6575-SUVOI7495-OOKFY4667-QONEA1658-BYGOY8769-KKTPH5067-XJQFU3957-SQXML5954-BDFSG0434-CSFON1215-FIGMP0526-SXDJP0557-ASKVU5698-BQJOH5122-CYFXA8627-ZYGSZ3410-YOQBQ3170-NMYVV7410-OIEDU2273-QCKXA0731-EBJQB1891-QRQSA8028-PNXHQ5630-AHZIL2673-CXQEU6416-JNVRK5590-ZSKRV0191-ZHSKZ3887-PBQAO5919-TMRGD8108-UXRQN9191-MYQKX9919-YLNNG0318-ZOPWS4445-ZJNMU4808-YNLEU1862-QRWPE2796-NNUFI3016-KSVCP4767-HZZHQ2424-WVHPL7233-PAHHL6067-NGCEU9848-JQJTX8637-XJIVR9803-FTJC

In [23]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [24]:
#Divisão do dataset em treino e testes
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [25]:
#Criação do modelo sequencial
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_shape=(X_train.shape[1],)))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
#Criação do modelo sequencial
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_shape=(X_train.shape[1],)))

In [30]:
#Compilação e treinamento do modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9894 - loss: 0.0529 - val_accuracy: 0.9912 - val_loss: 0.0518
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9877 - loss: 0.0486 - val_accuracy: 0.9912 - val_loss: 0.0515
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9920 - loss: 0.0459 - val_accuracy: 0.9912 - val_loss: 0.0516
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9848 - loss: 0.0754 - val_accuracy: 0.9912 - val_loss: 0.0518
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9746 - loss: 0.0948 - val_accuracy: 0.9912 - val_loss: 0.0521
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9922 - loss: 0.0453 - val_accuracy: 0.9912 - val_loss: 0.0520
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9728 - loss: 0.0895 - val_accuracy: 0.9912 - val_loss: 0.0518
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9911 - loss: 0.0511 - val_accuracy: 0.9912 - val_loss:

In [31]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


### Resultados

In [32]:
#Calculando a acurácia e F1-Score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)

Accuracy: 0.9912280701754386
F1 Score: 0.993006993006993


# Interpretação dos Resultados

1. **Acurácia: 99.12%**
   - O modelo tem uma precisão de 99.12% das previsões no conjunto de teste. Isso indica um desempenho muito bom, mas sozinho, não revela se o modelo está lidando adequadamente com todas as classes, especialmente em casos de desequilíbrio.

2. **F1-Score: 99.30%**
   - O F1-Score alto sugere que o modelo tem um bom equilíbrio entre precisão (quantos dos identificados como positivos realmente são positivos) e recall (quantos dos reais positivos foram identificados). Isso é especialmente relevante em conjuntos de dados com classes desbalanceadas.

## Desempenho Geral

- **Eficiência**: Ambos os resultados são excelentes, indicando que o modelo está pegando bem as características dos dados e classificando as instâncias com precisão.
- **Equilíbrio entre Classes**: O alto F1-Score sugere que o modelo está tratando bem tanto a classe majoritária quanto a minoritária.

##Possíveis Melhorias

1. **Resultados visuais**:
   - Trazer os resultados de uma maneira mais visual, através de graficos intuitivos.
2. **Ajustar Hiperparâmetros**:
   - Pequenos ajustes na arquitetura ou nos hiperparâmetros podem ainda trazer melhorias.

3. **Analisar Erros**:
   - Verificar onde o modelo erra pode oferecer insights para ajustes específicos.